In [ ]:
from contextlib import contextmanager
import sqlite3
import threading
@contextmanager
def get_connection(self):
    """Safe connection handler with automatic cleanup"""
    conn = None
    try:
        conn = sqlite3.connect('mydatabase.db', timeout=10)
        conn.execute("PRAGMA busy_timeout = 5000")  # 5 second timeout
        yield conn
    except sqlite3.Error as e:
        print(f"Database error: {e}")
        raise
    finally:
        if conn:
            conn.close()

In [ ]:
import sqlite3
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()
# edu=enduser/table structure
edutable=""" CREATE TABLE user (
            Userid INTEGER(25) NOT NULL PRIMARY KEY,
            First_Name CHAR(25) NOT NULL,
            Last_Name CHAR(25),
            Status INTEGER(25),
            Email VARCHAR(255),
            Password VARCHAR(255) NOT NULL,
            Fine INT
            ); """

#creates the table
cursor.execute(edutable)
cursor.close()
conn.close()

In [1]:
import sqlite3
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()
cursor.execute("DROP TABLE user")
conn.close()

In [1]:
import requests

response = requests.get("https://openlibrary.org/search.json?q=*&sort=rating&limit=100")
data = response.json()

In [24]:
import sqlite3
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()
cursor.execute("DROP TABLE book")
conn.close()

OperationalError: no such table: book

In [25]:
import sqlite3
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()
bktable=""" CREATE TABLE book (
            Bookid VARCHAR(255) NOT NULL PRIMARY KEY,
            Title CHAR(25) NOT NULL,
            Author CHAR(25),
            Status INTEGER(25),
            Description TEXT,
            Date TEXT,
            Img TEXT
            ); """

#creates the table
cursor.execute(bktable)
conn.close()

In [12]:
import os
APIKEY = os.getenv("API_KEY")
params = {
        "q":data["docs"][0]['title'], 
        "key":APIKEY,
    }
response = requests.get("https://www.googleapis.com/books/v1/volumes",params=params)
response.raise_for_status()
data2 = response.json()


In [23]:
data2['items'][0]['volumeInfo']['imageLinks']['thumbnail']

'http://books.google.com/books/content?id=KwhmCgAAQBAJ&printsec=frontcover&img=1&zoom=1&edge=curl&source=gbs_api'

In [26]:
import requests
import sqlite3
from dotenv import load_dotenv
import os
import requests

response = requests.get("https://openlibrary.org/search.json?q=*&sort=rating&limit=100")
data = response.json()
APIKEY = os.getenv("API_KEY")

conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()

j=0
for  i in data["docs"]:
    params = {
        "q": i["title"], 
        "key":APIKEY,
    }
    try:
        response = requests.get("https://www.googleapis.com/books/v1/volumes",params=params)
        response.raise_for_status()
        data2 = response.json()
        data3=data2['items'][0]['volumeInfo']
        query = f"""INSERT INTO book 
                    (Bookid, Title, Author, Status, Description, Date,Img) 
                    VALUES (?, ?, ?, ?, ?, ?, ?)"""
        values = (
        f'B{j}', 
        i["title"], 
        data3['authors'][0], 
        0, 
        data3['description'],
        data3['publishedDate'],
        data3['imageLinks']['thumbnail']
        )
        cursor.execute(query, values)
        conn.commit()
        j=j+1
    except:
        pass
conn.close()
    

In [2]:
import sqlite3
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()
btable = """
CREATE TABLE IF NOT EXISTS borrow (
    UserID VARCHAR(255) NOT NULL,
    BookID VARCHAR(255) NOT NULL,
    BorrowDate TEXT NOT NULL DEFAULT CURRENT_DATE,
    DueDate TEXT NOT NULL,
    PRIMARY KEY (UserID, BookID),
    FOREIGN KEY (UserID) REFERENCES user(UserID) ON DELETE CASCADE,
    FOREIGN KEY (BookID) REFERENCES book(BookID) ON DELETE CASCADE
);
"""
cursor.execute(btable)
conn.close()

In [1]:
import sqlite3
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()
cursor.execute("DROP TABLE borrow")
conn.close()